In [1]:
import os
import sys

from pathlib import Path

from copy import deepcopy

import numpy as np
import math

import torch
from torch import nn
import torch.nn.functional as F
import torchvision

from pytorch_model_summary import summary

In [2]:
params = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "save_pth": "checkpoint/",
    "reg_max": 4,
}
params['device'] = 'cpu'

## Build detection model

In [43]:
class ConvModule(nn.Module):
    """ 
    Convolutional block composed of conv->batchnorm->relu. 
    """
    def __init__(self, cin=1, cout=1, k=1, s=1, p=0, device='cpu'):
        super(ConvModule, self).__init__()
        self.conv = nn.Conv2d(cin, cout, (k, k), stride=s, padding=p, bias=False).to(device)
        self.bn = nn.BatchNorm2d(cout, eps=0.001, momentum=0.03, affine=True, track_running_stats=True).to(device)
        self.silu = nn.SiLU(inplace=True).to(device)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.silu(x)
        return x

In [44]:
class Bottleneck(nn.Module):
    """ 
    Bottleneck block componsed of conv->conv->residual connection. 
    """
    def __init__(self, c=1, shortcut=False, device='cpu'):
        super(Bottleneck, self).__init__()
        self.conv1 = ConvModule(cin=c, cout=c//2, k=3, s=1, p=1, device=device)
        self.conv2 = ConvModule(cin=c//2, cout=c, k=3, s=1, p=1, device=device)
        self.shortcut = shortcut
        
    def forward(self, x):
        xin = x
        x = self.conv1(x)
        x = self.conv2(x)
        if self.shortcut==True:
            x = xin + x
            return x
        return x

In [45]:
class C2f(nn.Module):
    """ 
    C2f module (cross-stage partial bottleneck with two convolutions) which combines 
    high-level features with contextual information to improve detection accuracy. 
    """
    def __init__(self, cin=1, cout=1, depth=1, device='cpu'):
        super(C2f, self).__init__()
        self.cout = cout
        self.depth = depth
        self.convmodule1 = ConvModule(cin=cin, cout=cout, k=1, s=1, p=0, device=device)
        self.bottleneck = []
        for _ in range(depth):
            self.bottleneck.append(Bottleneck(c=self.cout//2, shortcut=True, device=device))
        cin = cout//2 * (depth+2)
        self.convmodule2 = ConvModule(cin=cin, cout=cout, k=1, s=1, p=0, device=device)
        
    def forward(self, x):
        x1 = self.convmodule1(x)
        x1_1, x1_2 = torch.split(x1, self.cout//2, dim=1)
        x3 = torch.cat([x1_1, x1_2],dim=1)
        for i in range(len(self.bottleneck)):
            x2 = self.bottleneck[i](x1_2)
            x3 = torch.cat([x3, x2], dim=1)
            x1_2 = x2
        x = self.convmodule2(x3)
        return x

In [46]:
class SPPF(nn.Module):
    """ 
    Spatial pyramid pooling fast module (SPPF) layer accelerates computation 
    by pooling features into a fixed-size map. 
    """
    def __init__(self, c=1, device='cpu'):
        super(SPPF, self).__init__()
        self.conv1 = ConvModule(cin=c, cout=c, k=1, s=1, p=0, device=device)
        self.mp1 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.mp2 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.mp3 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.conv2 = ConvModule(cin=c*4, cout=c, k=1, s=1, p=0, device=device)

    def forward(self, x):
        x = self.conv1(x)
        x1 = self.mp1(x)
        x2 = self.mp2(x1)
        x3 = self.mp3(x2)
        x = torch.cat([x, x1, x2, x3], dim=1)
        x = self.conv2(x)
        return x

In [47]:
class DetectionHead(nn.Module):
    """
    Detection head module, which is decoupled to regression, classification, 
    and depth central pixel estimation tasks independently.
    """
    def __init__(self, c=1, reg_max=1, nclass=1, device='cpu'):
        super(DetectionHead, self).__init__()
        d = max(c, reg_max*4)
        self.bboxconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.bboxconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.bboxconv3 = nn.Conv2d(d, 4*reg_max, (1, 1), stride=1, padding=0, bias=False).to(device)
        self.clsconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.clsconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.clsconv3 = nn.Conv2d(d, nclass, (1, 1), stride=1, padding=0, bias=False).to(device)
        self.dptconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.dptconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.dptconv3 = nn.Conv2d(d, 1, (1, 1), stride=1, padding=0, bias=False).to(device)
      
    def forward(self, x):
        # bbox branch
        xbbox = self.bboxconv1(x)
        xbbox = self.bboxconv2(xbbox)
        xbbox = self.bboxconv3(xbbox)
        # cls branch
        xcls = self.clsconv1(x)
        xcls = self.clsconv2(xcls)
        xcls = self.clsconv3(xcls)
        # depth branch
        xdpt = self.dptconv1(x)
        xdpt = self.dptconv2(xdpt)
        xdpt = self.dptconv3(xdpt)
        
        feats = torch.cat([xbbox, xcls, xdpt], dim=1) 
        return feats

In [48]:
class DFL(nn.Module):
    """
    Integral module of Distribution Focal Loss (DFL).
    Proposed in Generalized Focal Loss https://ieeexplore.ieee.org/document/9792391
    """
    def __init__(self, c1=16, device='cpu'):
        """Initialize a convolutional layer with a given number of input channels."""
        super(DFL, self).__init__()
        self.conv = nn.Conv2d(c1, 1, 1, bias=False).requires_grad_(False).to(device)
        x = torch.arange(c1, dtype=torch.float)
        self.conv.weight.data[:] = nn.Parameter(x.view(1, c1, 1, 1))
        self.c1 = c1

    def forward(self, x):
        """Applies a transformer layer on input tensor 'x' and returns a tensor."""
        b, c, a = x.shape  # batch, channels, anchors
        return self.conv(x.view(b, 4, self.c1, a).transpose(2, 1).softmax(1)).view(b, 4, a)

In [49]:
class Inference(nn.Module):
    def __init__(self, nclasses=1, stride=None, reg_max=1, device='cpu'):
        super(Inference, self).__init__()
        self.stride = stride
        self.nc = nclasses
        self.reg_max = reg_max
        self.no = self.reg_max*4 + nclasses + 1
        self.dfl = DFL(self.reg_max, device=device) #if self.reg_max > 1 else nn.Identity()
        
    def forward(self, feats):
        # Extract predictions from each head at different strides
        pred_distri, pred_scores, pred_depth = torch.cat([xi.view(feats[0].shape[0], self.no, -1) for xi in feats], 2).split((self.reg_max*4, self.nc, 1), 1)
        pred_scores = pred_scores.permute(0, 1, 2).contiguous() # (b, nc, h*w)
        pred_distri = pred_distri.permute(0, 1, 2).contiguous() # (b, 4*reg_max, h*w)
        pred_depth = pred_depth.permute(0, 1, 2).contiguous() # (b, 1, h*w)
        # Get anchor point centers from output grids and its corresponding stride
        anchors, strides = (x.transpose(0, 1) for x in self.make_anchors(feats, self.stride, 0.5))
        # Decode reg_max*4 prediction to cxywh bounding box prediction
        dbox = self.dist2bbox(self.dfl(pred_distri), anchors.unsqueeze(0), xywh=True, dim=1).clamp_(0.) * strides
        y = torch.cat((dbox, pred_scores.sigmoid(), pred_depth), 1) # (bs, 4 + nclasses + depth, h*w)
        return y
    
    def dist2bbox(self, distance, anchor_points, xywh=True, dim=-1):
        """Transform distance(ltrb) to box(xywh or xyxy).
                width and height of bounding box are in range [0, 2*(self.reg_max-1)] owing to (x2y2-x1y1=rb+lt) 
        """
        lt, rb = distance.chunk(2, dim) # lt and rb is in range[0, self.reg_max-1] 
        x1y1 = anchor_points - lt
        x2y2 = anchor_points + rb
        if xywh:
            c_xy = (x1y1 + x2y2) / 2
            wh = x2y2 - x1y1
            return torch.cat((c_xy, wh), dim)  # xywh bbox
        return torch.cat((x1y1, x2y2), dim)  # xyxy bbox

    def make_anchors(self, feats, strides, grid_cell_offset=0.5):
        """Generate anchors from features."""
        anchor_points, stride_tensor = [], []
        assert feats is not None
        dtype, device = feats[0].dtype, feats[0].device
        for i, stride in enumerate(strides):
            _, _, h, w = feats[i].shape
            sx = torch.arange(end=w, device=device, dtype=dtype) + grid_cell_offset  # shift x
            sy = torch.arange(end=h, device=device, dtype=dtype) + grid_cell_offset  # shift y
            sy, sx = torch.meshgrid(sy, sx, indexing='ij')
            anchor_points.append(torch.stack((sx, sy), -1).view(-1, 2))
            stride_tensor.append(torch.full((h * w, 1), stride, dtype=dtype, device=device))
        return torch.cat(anchor_points), torch.cat(stride_tensor)

In [50]:
class ObjectDetector(nn.Module):
    """
    Object Detection model inspired on YOLOv8 from Ultralytics (https://docs.ultralytics.com/models/yolov8/#supported-tasks).
    The features maps has been divided by two respect the nano version, 
    in order to reduce model size for edge devices.
    The detection head incorportes a new feature: a decoupled head for 
    depth estimation of the central pixel of the regressed bounding boxes.
    
    Args:
        nclasses (int): number of classes in the classification task of bounding boxes.
        device (string): device to initiate and proccess weights; cpu or cuda.
    
    Attributes:
        convX (nn.Conv2d): two dimensional convolution layer to extract features along
                           different resolution maps.
        sppf (nn.Module): spatial pyramid pooling fast module.
        c2f_x (nn.Module): cross-stage partial bottleneck module.
        upsample (nn.Upsample): upsampling layer to concatenate features in the neck 
                                control connections.
        headX (nn.Module): detection head for different features resolution maps.
        
    Methods:
        forward(self, x): forward given input along detection model.
    """
    def __init__(self, nclasses=1, reg_max=1, device='cpu'):
        super(ObjectDetector, self).__init__()

        self.conv1 = ConvModule(cin=3, cout=16, k=3, s=2, p=1, device=device)
        self.conv2 = ConvModule(cin=16, cout=32, k=3, s=2, p=1, device=device)
        self.conv3 = ConvModule(cin=32, cout=64, k=3, s=2, p=1, device=device)
        self.conv4 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv5 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv6 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv7 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        
        self.sppf = SPPF(c=64, device=device)
        
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest').to(device)

        self.c2f_1 = C2f(cin=32, cout=32, depth=1, device=device)
        self.c2f_2 = C2f(cin=64, cout=64, depth=2, device=device)
        self.c2f_3 = C2f(cin=64, cout=64, depth=2, device=device)
        self.c2f_4 = C2f(cin=64, cout=64, depth=1, device=device)
        self.c2f_5 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_6 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_7 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_8 = C2f(cin=128, cout=64, depth=1, device=device)
        
        self.head1 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        self.head2 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        self.head3 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        
        #self.inference = Inference(nclasses=nclasses, stride=torch.tensor([8,16,32]), reg_max=reg_max, device=device)
        
    def forward(self, x):

        ## ------------------------------ BACKBONE ------------------------------------
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        c2f_1 = self.c2f_1(x2)
        x3 = self.conv3(c2f_1)
        c2f_2 = self.c2f_2(x3)
        x4 = self.conv4(c2f_2)
        c2f_3 = self.c2f_3(x4)
        x5 = self.conv5(c2f_3)
        c2f_4 = self.c2f_4(x5)
        sppf = self.sppf(c2f_4)
        
        ## ------------------------------ NECK ------------------------------------
        ## process branch
        up_1 = self.upsample(sppf)
        cat_1 = torch.cat([up_1, c2f_3], dim=1)
        c2f_5 = self.c2f_5(cat_1)      
        up_2 = self.upsample(c2f_5)    
        cat_2 = torch.cat([up_2, c2f_2], dim=1)
        c2f_6 = self.c2f_6(cat_2)

        ## error feedback branch
        x6 = self.conv6(c2f_6)
        cat_3 = torch.cat([x6, c2f_5], dim=1)
        c2f_7 = self.c2f_7(cat_3)
        x7 = self.conv7(c2f_7)
        cat_4 = torch.cat([x7, sppf], dim=1)
        c2f_8 = self.c2f_8(cat_4)
    
        ## ------------------------------ HEAD ----------------------------------
        head1 = self.head1(c2f_6)
        head2 = self.head2(c2f_7)
        head3 = self.head3(c2f_8)
        
        head_detections = (head1, head2, head3)
        #y = self.inference(head_detections)
        
        return head_detections

## Load pre-trained model

In [51]:
# Load Object Detection Trained model
if str(params["device"]) == 'cuda':
    trained_model = torch.load('checkpoint/coco_objdet.pt').to(params["device"])
else:
    trained_model = torch.load('checkpoint/coco_objdet.pt', map_location=torch.device('cpu')).to(params["device"])

# Load Depth Estimation Trained model    
if str(params["device"]) == 'cuda':
    dpt_model = torch.load('checkpoint/coco_depth.pt').to(params["device"])
else:
    dpt_model = torch.load('checkpoint/coco_depth.pt', map_location=torch.device('cpu')).to(params["device"])


# Copy Depth Estimation head weights from depth model to objdet model
trained_model.head1.dptconv1 = deepcopy(dpt_model.head1.dptconv1)
trained_model.head1.dptconv2 = deepcopy(dpt_model.head1.dptconv2)
trained_model.head1.dptconv3 = deepcopy(dpt_model.head1.dptconv3)

trained_model.head2.dptconv1 = deepcopy(dpt_model.head2.dptconv1)
trained_model.head2.dptconv2 = deepcopy(dpt_model.head2.dptconv2)
trained_model.head2.dptconv3 = deepcopy(dpt_model.head2.dptconv3)

trained_model.head3.dptconv1 = deepcopy(dpt_model.head3.dptconv1)
trained_model.head3.dptconv2 = deepcopy(dpt_model.head3.dptconv2)
trained_model.head3.dptconv3 = deepcopy(dpt_model.head3.dptconv3)

del dpt_model

for name, param in trained_model.named_parameters():
    param.requires_grad = False
    #param.grad = None

trained_model.eval()
print(summary(trained_model.to(params["device"]), torch.zeros((1, 3, 640, 640)).to(params["device"]), show_input=True))

--------------------------------------------------------------------------
       Layer (type)           Input Shape         Param #     Tr. Param #
       ConvModule-1      [1, 3, 640, 640]             464               0
       ConvModule-2     [1, 16, 320, 320]           4,672               0
              C2f-3     [1, 32, 160, 160]           2,688               0
       ConvModule-4     [1, 32, 160, 160]          18,560               0
              C2f-5       [1, 64, 80, 80]          12,544               0
       ConvModule-6       [1, 64, 80, 80]          36,992               0
              C2f-7       [1, 64, 40, 40]          12,544               0
       ConvModule-8       [1, 64, 40, 40]          36,992               0
              C2f-9       [1, 64, 20, 20]          10,496               0
            SPPF-10       [1, 64, 20, 20]          20,736               0
        Upsample-11       [1, 64, 20, 20]               0               0
             C2f-12      [1, 128, 40,

## Correct C2F module: change a list of tensors to be a class of nn.Sequential.

In [52]:
class ConvModule(nn.Module):
    """
    Convolutional block composed of conv->batchnorm->relu. 
    """
    def __init__(self, cin=1, cout=1, k=1, s=1, p=0, device='cpu'):
        super(ConvModule, self).__init__()
        self.conv = nn.Conv2d(cin, cout, (k, k), stride=s, padding=p, bias=False).to(device)
        self.bn = nn.BatchNorm2d(cout, eps=0.001, momentum=0.03, affine=True, track_running_stats=True).to(device)
        self.silu = nn.SiLU(inplace=True).to(device)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.silu(x)
        return x

In [53]:
class Bottleneck(nn.Module):
    """ 
    Bottleneck block componsed of conv->conv->residual connection. 
    """
    def __init__(self, c=1, shortcut=False, device='cpu'):
        super(Bottleneck, self).__init__()
        self.conv1 = ConvModule(cin=c, cout=c//2, k=3, s=1, p=1, device=device)
        self.conv2 = ConvModule(cin=c//2, cout=c, k=3, s=1, p=1, device=device)
        self.shortcut = shortcut
        
    def forward(self, x):
        xin = x
        x = self.conv1(x)
        x = self.conv2(x)
        if self.shortcut==True:
            x = xin + x
            return x
        return x

In [54]:
class C2f(nn.Module):
    """ 
    C2f module (cross-stage partial bottleneck with two convolutions) which combines 
    high-level features with contextual information to improve detection accuracy. 
    """
    def __init__(self, cin=1, cout=1, depth=1, device='cpu'):
        super(C2f, self).__init__()
        self.cout = cout
        self.depth = depth
        self.convmodule1 = ConvModule(cin=cin, cout=cout, k=1, s=1, p=0, device=device)
        bottleneck = []
        for _ in range(depth):
            bottleneck.append(Bottleneck(c=self.cout//2, shortcut=True, device=device))
        self.bottleneck = nn.Sequential(*bottleneck)
        cin = cout//2 * (depth+2)
        self.convmodule2 = ConvModule(cin=cin, cout=cout, k=1, s=1, p=0, device=device)
        
    def forward(self, x):
        x1 = self.convmodule1(x)
        x1_1, x1_2 = torch.split(x1, self.cout//2, dim=1)
        x3 = torch.cat([x1_1, x1_2],dim=1)
        for mod in self.bottleneck:
            x2 = mod(x1_2)
            x3 = torch.cat([x3, x2], dim=1)
            x1_2 = x2
        x = self.convmodule2(x3)
        return x

In [55]:
class SPPF(nn.Module):
    """ 
    Spatial pyramid pooling fast module (SPPF) layer accelerates computation 
    by pooling features into a fixed-size map. 
    """
    def __init__(self, c=1, device='cpu'):
        super(SPPF, self).__init__()
        self.conv1 = ConvModule(cin=c, cout=c, k=1, s=1, p=0, device=device)
        self.mp1 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.mp2 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.mp3 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False).to(device)
        self.conv2 = ConvModule(cin=c*4, cout=c, k=1, s=1, p=0, device=device)

    def forward(self, x):
        x = self.conv1(x)
        x1 = self.mp1(x)
        x2 = self.mp2(x1)
        x3 = self.mp3(x2)
        x = torch.cat([x, x1, x2, x3], dim=1)
        x = self.conv2(x)
        return x

In [56]:
class DetectionHead(nn.Module):
    """
    Detection head module, which is decoupled to regression, classification, 
    and depth central pixel estimation tasks independently.
    """
    def __init__(self, c=1, reg_max=1, nclass=1, device='cpu'):
        super(DetectionHead, self).__init__()
        d = max(c, reg_max*4)
        self.bboxconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.bboxconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.bboxconv3 = nn.Conv2d(d, 4*reg_max, (1, 1), stride=1, padding=0, bias=False).to(device)
        self.clsconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.clsconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.clsconv3 = nn.Conv2d(d, nclass, (1, 1), stride=1, padding=0, bias=False).to(device)
        self.dptconv1 = ConvModule(cin=c, cout=d, k=3, s=1, p=1, device=device)
        self.dptconv2 = ConvModule(cin=d, cout=d, k=3, s=1, p=1, device=device)
        self.dptconv3 = nn.Conv2d(d, 1, (1, 1), stride=1, padding=0, bias=False).to(device)
      
    def forward(self, x):
        # bbox branch
        xbbox = self.bboxconv1(x)
        xbbox = self.bboxconv2(xbbox)
        xbbox = self.bboxconv3(xbbox)
        # cls branch
        xcls = self.clsconv1(x)
        xcls = self.clsconv2(xcls)
        xcls = self.clsconv3(xcls)
        # depth branch
        xdpt = self.dptconv1(x)
        xdpt = self.dptconv2(xdpt)
        xdpt = self.dptconv3(xdpt)
        
        feats = torch.cat([xbbox, xcls, xdpt], dim=1) 
        return feats

In [57]:
class ObjectDetector(nn.Module):
    """
    Object Detection model inspired on YOLOv8 from Ultralytics (https://docs.ultralytics.com/models/yolov8/#supported-tasks).
    The features maps has been divided by two respect the nano version, 
    in order to reduce model size for edge devices.
    The detection head incorportes a new feature: a decoupled head for 
    depth estimation of the central pixel of the regressed bounding boxes.
    
    Args:
        nclasses (int): number of classes in the classification task of bounding boxes.
        device (string): device to initiate and proccess weights; cpu or cuda.
    
    Attributes:
        convX (nn.Conv2d): two dimensional convolution layer to extract features along
                           different resolution maps.
        sppf (nn.Module): spatial pyramid pooling fast module.
        c2f_x (nn.Module): cross-stage partial bottleneck module.
        upsample (nn.Upsample): upsampling layer to concatenate features in the neck 
                                control connections.
        headX (nn.Module): detection head for different features resolution maps.
        
    Methods:
        forward(self, x): forward given input along detection model.
    """
    def __init__(self, nclasses=1, reg_max=1, device='cpu'):
        super(ObjectDetector, self).__init__()

        self.conv1 = ConvModule(cin=3, cout=16, k=3, s=2, p=1, device=device)
        self.conv2 = ConvModule(cin=16, cout=32, k=3, s=2, p=1, device=device)
        self.conv3 = ConvModule(cin=32, cout=64, k=3, s=2, p=1, device=device)
        self.conv4 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv5 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv6 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        self.conv7 = ConvModule(cin=64, cout=64, k=3, s=2, p=1, device=device)
        
        self.sppf = SPPF(c=64, device=device)
        
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest').to(device)

        self.c2f_1 = C2f(cin=32, cout=32, depth=1, device=device)
        self.c2f_2 = C2f(cin=64, cout=64, depth=2, device=device)
        self.c2f_3 = C2f(cin=64, cout=64, depth=2, device=device)
        self.c2f_4 = C2f(cin=64, cout=64, depth=1, device=device)
        self.c2f_5 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_6 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_7 = C2f(cin=128, cout=64, depth=1, device=device)
        self.c2f_8 = C2f(cin=128, cout=64, depth=1, device=device)
        
        self.head1 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        self.head2 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        self.head3 = DetectionHead(c=64, reg_max=reg_max, nclass=nclasses, device=device)
        
        #self.inference = Inference(nclasses=nclasses, stride=torch.tensor([8,16,32]), reg_max=reg_max, device=device)
        
    def forward(self, x):

        ## ------------------------------ BACKBONE ------------------------------------
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        c2f_1 = self.c2f_1(x2)
        x3 = self.conv3(c2f_1)
        c2f_2 = self.c2f_2(x3)
        x4 = self.conv4(c2f_2)
        c2f_3 = self.c2f_3(x4)
        x5 = self.conv5(c2f_3)
        c2f_4 = self.c2f_4(x5)
        sppf = self.sppf(c2f_4)
        
        ## ------------------------------ NECK ------------------------------------
        ## process branch
        up_1 = self.upsample(sppf)
        cat_1 = torch.cat([up_1, c2f_3], dim=1)
        c2f_5 = self.c2f_5(cat_1)      
        up_2 = self.upsample(c2f_5)    
        cat_2 = torch.cat([up_2, c2f_2], dim=1)
        c2f_6 = self.c2f_6(cat_2)

        ## error feedback branch
        x6 = self.conv6(c2f_6)
        cat_3 = torch.cat([x6, c2f_5], dim=1)
        c2f_7 = self.c2f_7(cat_3)
        x7 = self.conv7(c2f_7)
        cat_4 = torch.cat([x7, sppf], dim=1)
        c2f_8 = self.c2f_8(cat_4)
    
        ## ------------------------------ HEAD ----------------------------------
        head1 = self.head1(c2f_6)
        head2 = self.head2(c2f_7)
        head3 = self.head3(c2f_8)
        
        head_detections = (head1, head2, head3)
        #y = self.inference(head_detections)
        
        return head_detections

In [58]:
new_model = ObjectDetector(nclasses=nclasses, reg_max=params["reg_max"], device=params["device"])

for name, param in new_model.named_parameters():
    param.requires_grad = False
    #param.grad = None

#model.to(torch.float32).eval()
new_model.eval()
print(summary(new_model.to(params["device"]), torch.zeros((1, 3, 640, 640)).to(params["device"]), show_input=True))

--------------------------------------------------------------------------
       Layer (type)           Input Shape         Param #     Tr. Param #
       ConvModule-1      [1, 3, 640, 640]             464               0
       ConvModule-2     [1, 16, 320, 320]           4,672               0
              C2f-3     [1, 32, 160, 160]           5,040               0
       ConvModule-4     [1, 32, 160, 160]          18,560               0
              C2f-5       [1, 64, 80, 80]          31,168               0
       ConvModule-6       [1, 64, 80, 80]          36,992               0
              C2f-7       [1, 64, 40, 40]          31,168               0
       ConvModule-8       [1, 64, 40, 40]          36,992               0
              C2f-9       [1, 64, 20, 20]          19,808               0
            SPPF-10       [1, 64, 20, 20]          20,736               0
        Upsample-11       [1, 64, 20, 20]               0               0
             C2f-12      [1, 128, 40,

In [59]:
# Copy all weights from model to the new model with the C2F module corrected (the arch doesnt change at all)

# conv1
new_model.conv1.conv = deepcopy(trained_model.conv1.conv)
new_model.conv1.bn = deepcopy(trained_model.conv1.bn)
new_model.conv1.silu = deepcopy(trained_model.conv1.silu)
# conv2
new_model.conv2.conv = deepcopy(trained_model.conv2.conv)
new_model.conv2.bn = deepcopy(trained_model.conv2.bn)
new_model.conv2.silu = deepcopy(trained_model.conv2.silu)
# conv3
new_model.conv3.conv = deepcopy(trained_model.conv3.conv)
new_model.conv3.bn = deepcopy(trained_model.conv3.bn)
new_model.conv3.silu = deepcopy(trained_model.conv3.silu)
# conv4
new_model.conv4.conv = deepcopy(trained_model.conv4.conv)
new_model.conv4.bn = deepcopy(trained_model.conv4.bn)
new_model.conv4.silu = deepcopy(trained_model.conv4.silu)
# conv5
new_model.conv5.conv = deepcopy(trained_model.conv5.conv)
new_model.conv5.bn = deepcopy(trained_model.conv5.bn)
new_model.conv5.silu = deepcopy(trained_model.conv5.silu)
# conv6
new_model.conv6.conv = deepcopy(trained_model.conv6.conv)
new_model.conv6.bn = deepcopy(trained_model.conv6.bn)
new_model.conv6.silu = deepcopy(trained_model.conv6.silu)
# conv7
new_model.conv7.conv = deepcopy(trained_model.conv7.conv)
new_model.conv7.bn = deepcopy(trained_model.conv7.bn)
new_model.conv7.silu = deepcopy(trained_model.conv7.silu)
# spff
new_model.sppf.conv1.conv = deepcopy(trained_model.sppf.conv1.conv)
new_model.sppf.conv1.bn = deepcopy(trained_model.sppf.conv1.bn)
new_model.sppf.conv1.silu = deepcopy(trained_model.sppf.conv1.silu)
new_model.sppf.mp1 = deepcopy(trained_model.sppf.mp1)
new_model.sppf.mp2 = deepcopy(trained_model.sppf.mp2)
new_model.sppf.mp3 = deepcopy(trained_model.sppf.mp3)
new_model.sppf.conv2.conv = deepcopy(trained_model.sppf.conv2.conv)
new_model.sppf.conv2.bn = deepcopy(trained_model.sppf.conv2.bn)
new_model.sppf.conv2.silu = deepcopy(trained_model.sppf.conv2.silu)
# c2f modules 
# c2f_1
new_model.c2f_1.convmodule1.conv = deepcopy(trained_model.c2f_1.convmodule1.conv)
new_model.c2f_1.convmodule1.bn = deepcopy(trained_model.c2f_1.convmodule1.bn)
new_model.c2f_1.convmodule1.silu = deepcopy(trained_model.c2f_1.convmodule1.silu)
new_model.c2f_1.convmodule2.conv = deepcopy(trained_model.c2f_1.convmodule2.conv)
new_model.c2f_1.convmodule2.bn = deepcopy(trained_model.c2f_1.convmodule2.bn)
new_model.c2f_1.convmodule2.silu = deepcopy(trained_model.c2f_1.convmodule2.silu)

new_model.c2f_1.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_1.bottleneck[0].conv1.conv)
new_model.c2f_1.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_1.bottleneck[0].conv1.bn)
new_model.c2f_1.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_1.bottleneck[0].conv1.silu)

new_model.c2f_1.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_1.bottleneck[0].conv2.conv)
new_model.c2f_1.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_1.bottleneck[0].conv2.bn)
new_model.c2f_1.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_1.bottleneck[0].conv2.silu)
# c2f_2
new_model.c2f_2.convmodule1.conv = deepcopy(trained_model.c2f_2.convmodule1.conv)
new_model.c2f_2.convmodule1.bn = deepcopy(trained_model.c2f_2.convmodule1.bn)
new_model.c2f_2.convmodule1.silu = deepcopy(trained_model.c2f_2.convmodule1.silu)
new_model.c2f_2.convmodule2.conv = deepcopy(trained_model.c2f_2.convmodule2.conv)
new_model.c2f_2.convmodule2.bn = deepcopy(trained_model.c2f_2.convmodule2.bn)
new_model.c2f_2.convmodule2.silu = deepcopy(trained_model.c2f_2.convmodule2.silu)

new_model.c2f_2.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_2.bottleneck[0].conv1.conv)
new_model.c2f_2.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_2.bottleneck[0].conv1.bn)
new_model.c2f_2.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_2.bottleneck[0].conv1.silu)

new_model.c2f_2.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_2.bottleneck[0].conv2.conv)
new_model.c2f_2.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_2.bottleneck[0].conv2.bn)
new_model.c2f_2.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_2.bottleneck[0].conv2.silu)

new_model.c2f_2.bottleneck[1].conv1.conv = deepcopy(trained_model.c2f_2.bottleneck[1].conv1.conv)
new_model.c2f_2.bottleneck[1].conv1.bn = deepcopy(trained_model.c2f_2.bottleneck[1].conv1.bn)
new_model.c2f_2.bottleneck[1].conv1.silu = deepcopy(trained_model.c2f_2.bottleneck[1].conv1.silu)

new_model.c2f_2.bottleneck[1].conv2.conv = deepcopy(trained_model.c2f_2.bottleneck[1].conv2.conv)
new_model.c2f_2.bottleneck[1].conv2.bn = deepcopy(trained_model.c2f_2.bottleneck[1].conv2.bn)
new_model.c2f_2.bottleneck[1].conv2.silu = deepcopy(trained_model.c2f_2.bottleneck[1].conv2.silu)
# c2f_3
new_model.c2f_3.convmodule1.conv = deepcopy(trained_model.c2f_3.convmodule1.conv)
new_model.c2f_3.convmodule1.bn = deepcopy(trained_model.c2f_3.convmodule1.bn)
new_model.c2f_3.convmodule1.silu = deepcopy(trained_model.c2f_3.convmodule1.silu)
new_model.c2f_3.convmodule2.conv = deepcopy(trained_model.c2f_3.convmodule2.conv)
new_model.c2f_3.convmodule2.bn = deepcopy(trained_model.c2f_3.convmodule2.bn)
new_model.c2f_3.convmodule2.silu = deepcopy(trained_model.c2f_3.convmodule2.silu)

new_model.c2f_3.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_3.bottleneck[0].conv1.conv)
new_model.c2f_3.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_3.bottleneck[0].conv1.bn)
new_model.c2f_3.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_3.bottleneck[0].conv1.silu)

new_model.c2f_3.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_3.bottleneck[0].conv2.conv)
new_model.c2f_3.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_3.bottleneck[0].conv2.bn)
new_model.c2f_3.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_3.bottleneck[0].conv2.silu)

new_model.c2f_3.bottleneck[1].conv1.conv = deepcopy(trained_model.c2f_3.bottleneck[1].conv1.conv)
new_model.c2f_3.bottleneck[1].conv1.bn = deepcopy(trained_model.c2f_3.bottleneck[1].conv1.bn)
new_model.c2f_3.bottleneck[1].conv1.silu = deepcopy(trained_model.c2f_3.bottleneck[1].conv1.silu)

new_model.c2f_3.bottleneck[1].conv2.conv = deepcopy(trained_model.c2f_3.bottleneck[1].conv2.conv)
new_model.c2f_3.bottleneck[1].conv2.bn = deepcopy(trained_model.c2f_3.bottleneck[1].conv2.bn)
new_model.c2f_3.bottleneck[1].conv2.silu = deepcopy(trained_model.c2f_3.bottleneck[1].conv2.silu)
# c2f_4
new_model.c2f_4.convmodule1.conv = deepcopy(trained_model.c2f_4.convmodule1.conv)
new_model.c2f_4.convmodule1.bn = deepcopy(trained_model.c2f_4.convmodule1.bn)
new_model.c2f_4.convmodule1.silu = deepcopy(trained_model.c2f_4.convmodule1.silu)
new_model.c2f_4.convmodule2.conv = deepcopy(trained_model.c2f_4.convmodule2.conv)
new_model.c2f_4.convmodule2.bn = deepcopy(trained_model.c2f_4.convmodule2.bn)
new_model.c2f_4.convmodule2.silu = deepcopy(trained_model.c2f_4.convmodule2.silu)

new_model.c2f_4.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_4.bottleneck[0].conv1.conv)
new_model.c2f_4.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_4.bottleneck[0].conv1.bn)
new_model.c2f_4.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_4.bottleneck[0].conv1.silu)

new_model.c2f_4.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_4.bottleneck[0].conv2.conv)
new_model.c2f_4.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_4.bottleneck[0].conv2.bn)
new_model.c2f_4.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_4.bottleneck[0].conv2.silu)
# c2f_5
new_model.c2f_5.convmodule1.conv = deepcopy(trained_model.c2f_5.convmodule1.conv)
new_model.c2f_5.convmodule1.bn = deepcopy(trained_model.c2f_5.convmodule1.bn)
new_model.c2f_5.convmodule1.silu = deepcopy(trained_model.c2f_5.convmodule1.silu)
new_model.c2f_5.convmodule2.conv = deepcopy(trained_model.c2f_5.convmodule2.conv)
new_model.c2f_5.convmodule2.bn = deepcopy(trained_model.c2f_5.convmodule2.bn)
new_model.c2f_5.convmodule2.silu = deepcopy(trained_model.c2f_5.convmodule2.silu)

new_model.c2f_5.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_5.bottleneck[0].conv1.conv)
new_model.c2f_5.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_5.bottleneck[0].conv1.bn)
new_model.c2f_5.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_5.bottleneck[0].conv1.silu)

new_model.c2f_5.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_5.bottleneck[0].conv2.conv)
new_model.c2f_5.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_5.bottleneck[0].conv2.bn)
new_model.c2f_5.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_5.bottleneck[0].conv2.silu)
# c2f_6
new_model.c2f_6.convmodule1.conv = deepcopy(trained_model.c2f_6.convmodule1.conv)
new_model.c2f_6.convmodule1.bn = deepcopy(trained_model.c2f_6.convmodule1.bn)
new_model.c2f_6.convmodule1.silu = deepcopy(trained_model.c2f_6.convmodule1.silu)
new_model.c2f_6.convmodule2.conv = deepcopy(trained_model.c2f_6.convmodule2.conv)
new_model.c2f_6.convmodule2.bn = deepcopy(trained_model.c2f_6.convmodule2.bn)
new_model.c2f_6.convmodule2.silu = deepcopy(trained_model.c2f_6.convmodule2.silu)

new_model.c2f_6.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_6.bottleneck[0].conv1.conv)
new_model.c2f_6.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_6.bottleneck[0].conv1.bn)
new_model.c2f_6.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_6.bottleneck[0].conv1.silu)

new_model.c2f_6.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_6.bottleneck[0].conv2.conv)
new_model.c2f_6.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_6.bottleneck[0].conv2.bn)
new_model.c2f_6.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_6.bottleneck[0].conv2.silu)
# c2f_7
new_model.c2f_7.convmodule1.conv = deepcopy(trained_model.c2f_7.convmodule1.conv)
new_model.c2f_7.convmodule1.bn = deepcopy(trained_model.c2f_7.convmodule1.bn)
new_model.c2f_7.convmodule1.silu = deepcopy(trained_model.c2f_7.convmodule1.silu)
new_model.c2f_7.convmodule2.conv = deepcopy(trained_model.c2f_7.convmodule2.conv)
new_model.c2f_7.convmodule2.bn = deepcopy(trained_model.c2f_7.convmodule2.bn)
new_model.c2f_7.convmodule2.silu = deepcopy(trained_model.c2f_7.convmodule2.silu)

new_model.c2f_7.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_7.bottleneck[0].conv1.conv)
new_model.c2f_7.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_7.bottleneck[0].conv1.bn)
new_model.c2f_7.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_7.bottleneck[0].conv1.silu)

new_model.c2f_7.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_7.bottleneck[0].conv2.conv)
new_model.c2f_7.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_7.bottleneck[0].conv2.bn)
new_model.c2f_7.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_7.bottleneck[0].conv2.silu)
# c2f_8
new_model.c2f_8.convmodule1.conv = deepcopy(trained_model.c2f_8.convmodule1.conv)
new_model.c2f_8.convmodule1.bn = deepcopy(trained_model.c2f_8.convmodule1.bn)
new_model.c2f_8.convmodule1.silu = deepcopy(trained_model.c2f_8.convmodule1.silu)
new_model.c2f_8.convmodule2.conv = deepcopy(trained_model.c2f_8.convmodule2.conv)
new_model.c2f_8.convmodule2.bn = deepcopy(trained_model.c2f_8.convmodule2.bn)
new_model.c2f_8.convmodule2.silu = deepcopy(trained_model.c2f_8.convmodule2.silu)

new_model.c2f_8.bottleneck[0].conv1.conv = deepcopy(trained_model.c2f_8.bottleneck[0].conv1.conv)
new_model.c2f_8.bottleneck[0].conv1.bn = deepcopy(trained_model.c2f_8.bottleneck[0].conv1.bn)
new_model.c2f_8.bottleneck[0].conv1.silu = deepcopy(trained_model.c2f_8.bottleneck[0].conv1.silu)

new_model.c2f_8.bottleneck[0].conv2.conv = deepcopy(trained_model.c2f_8.bottleneck[0].conv2.conv)
new_model.c2f_8.bottleneck[0].conv2.bn = deepcopy(trained_model.c2f_8.bottleneck[0].conv2.bn)
new_model.c2f_8.bottleneck[0].conv2.silu = deepcopy(trained_model.c2f_8.bottleneck[0].conv2.silu)
# head1
new_model.head1.bboxconv1.conv = deepcopy(trained_model.head1.bboxconv1.conv)
new_model.head1.bboxconv1.bn = deepcopy(trained_model.head1.bboxconv1.bn)
new_model.head1.bboxconv1.silu = deepcopy(trained_model.head1.bboxconv1.silu)
new_model.head1.bboxconv2.conv = deepcopy(trained_model.head1.bboxconv2.conv)
new_model.head1.bboxconv2.bn = deepcopy(trained_model.head1.bboxconv2.bn)
new_model.head1.bboxconv2.silu = deepcopy(trained_model.head1.bboxconv2.silu)
new_model.head1.bboxconv3 = deepcopy(trained_model.head1.bboxconv3)

new_model.head1.clsconv1.conv = deepcopy(trained_model.head1.clsconv1.conv)
new_model.head1.clsconv1.bn = deepcopy(trained_model.head1.clsconv1.bn)
new_model.head1.clsconv1.silu = deepcopy(trained_model.head1.clsconv1.silu)
new_model.head1.clsconv2.conv = deepcopy(trained_model.head1.clsconv2.conv)
new_model.head1.clsconv2.bn = deepcopy(trained_model.head1.clsconv2.bn)
new_model.head1.clsconv2.silu = deepcopy(trained_model.head1.clsconv2.silu)
new_model.head1.clsconv3 = deepcopy(trained_model.head1.clsconv3)

new_model.head1.dptconv1.conv = deepcopy(trained_model.head1.dptconv1.conv)
new_model.head1.dptconv1.bn = deepcopy(trained_model.head1.dptconv1.bn)
new_model.head1.dptconv1.silu = deepcopy(trained_model.head1.dptconv1.silu)
new_model.head1.dptconv2.conv = deepcopy(trained_model.head1.dptconv2.conv)
new_model.head1.dptconv2.bn = deepcopy(trained_model.head1.dptconv2.bn)
new_model.head1.dptconv2.silu = deepcopy(trained_model.head1.dptconv2.silu)
new_model.head1.dptconv3 = deepcopy(trained_model.head1.dptconv3)
# head2
new_model.head2.bboxconv1.conv = deepcopy(trained_model.head2.bboxconv1.conv)
new_model.head2.bboxconv1.bn = deepcopy(trained_model.head2.bboxconv1.bn)
new_model.head2.bboxconv1.silu = deepcopy(trained_model.head2.bboxconv1.silu)
new_model.head2.bboxconv2.conv = deepcopy(trained_model.head2.bboxconv2.conv)
new_model.head2.bboxconv2.bn = deepcopy(trained_model.head2.bboxconv2.bn)
new_model.head2.bboxconv2.silu = deepcopy(trained_model.head2.bboxconv2.silu)
new_model.head2.bboxconv3 = deepcopy(trained_model.head2.bboxconv3)

new_model.head2.clsconv1.conv = deepcopy(trained_model.head2.clsconv1.conv)
new_model.head2.clsconv1.bn = deepcopy(trained_model.head2.clsconv1.bn)
new_model.head2.clsconv1.silu = deepcopy(trained_model.head2.clsconv1.silu)
new_model.head2.clsconv2.conv = deepcopy(trained_model.head2.clsconv2.conv)
new_model.head2.clsconv2.bn = deepcopy(trained_model.head2.clsconv2.bn)
new_model.head2.clsconv2.silu = deepcopy(trained_model.head2.clsconv2.silu)
new_model.head2.clsconv3 = deepcopy(trained_model.head2.clsconv3)

new_model.head2.dptconv1.conv = deepcopy(trained_model.head2.dptconv1.conv)
new_model.head2.dptconv1.bn = deepcopy(trained_model.head2.dptconv1.bn)
new_model.head2.dptconv1.silu = deepcopy(trained_model.head2.dptconv1.silu)
new_model.head2.dptconv2.conv = deepcopy(trained_model.head2.dptconv2.conv)
new_model.head2.dptconv2.bn = deepcopy(trained_model.head2.dptconv2.bn)
new_model.head2.dptconv2.silu = deepcopy(trained_model.head2.dptconv2.silu)
new_model.head2.dptconv3 = deepcopy(trained_model.head2.dptconv3)
# head3
new_model.head3.bboxconv1.conv = deepcopy(trained_model.head3.bboxconv1.conv)
new_model.head3.bboxconv1.bn = deepcopy(trained_model.head3.bboxconv1.bn)
new_model.head3.bboxconv1.silu = deepcopy(trained_model.head3.bboxconv1.silu)
new_model.head3.bboxconv2.conv = deepcopy(trained_model.head3.bboxconv2.conv)
new_model.head3.bboxconv2.bn = deepcopy(trained_model.head3.bboxconv2.bn)
new_model.head3.bboxconv2.silu = deepcopy(trained_model.head3.bboxconv2.silu)
new_model.head3.bboxconv3 = deepcopy(trained_model.head3.bboxconv3)

new_model.head3.clsconv1.conv = deepcopy(trained_model.head3.clsconv1.conv)
new_model.head3.clsconv1.bn = deepcopy(trained_model.head3.clsconv1.bn)
new_model.head3.clsconv1.silu = deepcopy(trained_model.head3.clsconv1.silu)
new_model.head3.clsconv2.conv = deepcopy(trained_model.head3.clsconv2.conv)
new_model.head3.clsconv2.bn = deepcopy(trained_model.head3.clsconv2.bn)
new_model.head3.clsconv2.silu = deepcopy(trained_model.head3.clsconv2.silu)
new_model.head3.clsconv3 = deepcopy(trained_model.head3.clsconv3)

new_model.head3.dptconv1.conv = deepcopy(trained_model.head3.dptconv1.conv)
new_model.head3.dptconv1.bn = deepcopy(trained_model.head3.dptconv1.bn)
new_model.head3.dptconv1.silu = deepcopy(trained_model.head3.dptconv1.silu)
new_model.head3.dptconv2.conv = deepcopy(trained_model.head3.dptconv2.conv)
new_model.head3.dptconv2.bn = deepcopy(trained_model.head3.dptconv2.bn)
new_model.head3.dptconv2.silu = deepcopy(trained_model.head3.dptconv2.silu)
new_model.head3.dptconv3 = deepcopy(trained_model.head3.dptconv3)

## Save merged model

In [63]:
# Save quantized model
version = params['version']
save_pth = os.getcwd() + '/' + params["save_pth"]
model_to_save = deepcopy(new_model.to('cpu'))
pth = Path(save_pth + f'/DOD_coco.pt')
torch.save(model_to_save, pth)